In [19]:
import inspect

class OverloadedFunction(list):
    '''
        wrapper around list of functions
        enable us to call it with arguments and it finds best match
    '''
    def __init__(self, f, name):
        super().__init__()
        self.append(f)
        self.name = name
        self.caller = None
    
    def __call__(self, *args, **kwargs):
        for func in self:
            sig = inspect.signature(func)
            try:
                bind = sig.bind(self.caller, *args, **kwargs)
                for key in bind.arguments:
                    if not sig.parameters[key].annotation == inspect.Parameter.empty:
                        if not isinstance(bind.arguments[key], sig.parameters[key].annotation):
                            break
                else:
                    return func(self.caller, *args, **kwargs)
            except:
                pass           
    
    def __get__(self, instance, cls):
        self.caller = instance
        return cls.__dict__[self.name]
    
    def __repr__(self):
        return '<overloaded function> signatures: ' \
                + str([inspect.signature(func) for func in self])

class OverloadingDict(dict):
    def __setitem__(self, key, value):
        if callable(value):
            if key in self:
                if not isinstance(self[key], list):
                    super().__setitem__(key, OverloadedFunction(self[key], key))
                self[key].append(value)
            else:
                super().__setitem__(key, value)
        else:
            super().__setitem__(key, value)

            
class OverloadingMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        return OverloadingDict()
    
    def __new__(cls, clsname, bases, classdict):
#         for key in classdict:
#             if isinstance(classdict[key], OverloadedFunc):
#                 classdict[key].sort
        return super().__new__(cls, clsname, bases, dict(classdict))
    
class OverloadingClass(metaclass=OverloadingMeta):
    pass
    

In [20]:
class Spam(OverloadingClass):
    def __init__(self, name):
        self.name = name
        
    def foo(self):
        print(f'{self.name}: no arguments')
        
    def foo(self, x:int):
        print(f'{self.name}: integer func {x}')
        
    def foo(self, x:float):
        print(f'{self.name}: function with float {x}')
        
    def foo(self, x:str, y:str):
        print(f'{self.name}: {x} {y}')

In [21]:
print()
s = Spam('GoodNameForClass')
s.foo()
s.foo(1)
s.foo(1.2)
s.foo('blabla', 'bla')


GoodNameForClass: no arguments
GoodNameForClass: integer func 1
GoodNameForClass: function with float 1.2
GoodNameForClass: blabla bla


In [23]:
30.75*12

369.0